In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

# importo los .py
import sys
sys.path.append("auxiliares")
import config
import feature_eng

from warnings import simplefilter 
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning)

In [2]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/Users/herna/labo3_empresa3_repo/")
#os.chdir("C:/diego_tools/labo3/")
os.chdir("/home/dcastro_contacto/buckets/b1/")
####################################################

In [3]:
#Manual
prefijo_arch_entrada = "datasets/emp3_sellout_base_cluster_manual_"
prefijo_arch_salida = "datasets/emp3_sellout_fe_cluster_manual_"
num_clusters=94

#Time Warping
#prefijo_arch_entrada = "datasets/emp3_sellout_base_cluster_warp_"
#prefijo_arch_salida = "datasets/emp3_sellout_fe_cluster_warp_"

arch_productos = "datasets/emp3_sellout_producto_base.csv"
arch_clientes = "datasets/emp3_sellout_cliente_base.csv"
arch_cat1 = "datasets/emp3_sellout_cat1_base.csv"
arch_cat2 = "datasets/emp3_sellout_cat2_base.csv"
arch_cat3 = "datasets/emp3_sellout_cat3_base.csv"

In [4]:
df_productos = pd.read_csv(arch_productos)
df_productos = df_productos.rename(columns={"product_id_tn":"tn"})
df_productos = df_productos[["product_id","periodo","tn"]] # limito solamente a columnas de interes
df_productos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34707 entries, 0 to 34706
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   product_id  34707 non-null  int64  
 1   periodo     34707 non-null  int64  
 2   tn          34707 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 813.6 KB


In [5]:
df_clientes = pd.read_csv(arch_clientes)
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18975 entries, 0 to 18974
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   customer_id     18975 non-null  int64  
 1   periodo         18975 non-null  int64  
 2   customer_id_tn  18975 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 444.9 KB


In [6]:
df_cat1 = pd.read_csv(arch_cat1)
df_cat1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cat1     140 non-null    object 
 1   periodo  140 non-null    int64  
 2   cat1_tn  140 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.4+ KB


In [7]:
df_cat2 = pd.read_csv(arch_cat2)
df_cat2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560 entries, 0 to 559
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cat2     560 non-null    object 
 1   periodo  560 non-null    int64  
 2   cat2_tn  560 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 13.2+ KB


In [8]:
df_cat3 = pd.read_csv(arch_cat3)
df_cat3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2717 entries, 0 to 2716
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cat3     2717 non-null   object 
 1   periodo  2717 non-null   int64  
 2   cat3_tn  2717 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 63.8+ KB


In [ ]:
for i in range(0,num_clusters):
    print("Cluster", i)
    
    df_sellout = pd.read_csv(prefijo_arch_entrada + str(i) + ".csv")
    
    #Variable Target
    df_sellout = feature_eng.crear_target_prod_cust(df_sellout)
    
    #Nuevos Features
    df_sellout = feature_eng.crear_nuevos_features(df_sellout)
    
    #Features temporales
    df_sellout = feature_eng.crear_features_temporales("prod_cust",df_sellout, config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)
    df_productos = feature_eng.crear_features_temporales("product_id",df_productos,config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)
    df_clientes = feature_eng.crear_features_temporales("customer_id",df_clientes,config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)
    df_cat1 = feature_eng.crear_features_temporales("cat1",df_cat1,config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)
    df_cat2 = feature_eng.crear_features_temporales("cat2",df_cat2, config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)
    df_cat3 = feature_eng.crear_features_temporales("cat3",df_cat3, config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM)

    #Joins
    print(len(df_sellout))
    df_productos_para_merge = df_productos.rename(columns={"tn":"product_id_tn"}) #revierto cambio para evitar colisiones
    
    df_sellout = df_sellout.merge(df_productos_para_merge,how="left",on=["product_id","periodo"])
    df_sellout = df_sellout.merge(df_clientes,how="left",on=["customer_id","periodo"])
    df_sellout = df_sellout.merge(df_cat1,how="left",on=["cat1","periodo"])
    df_sellout = df_sellout.merge(df_cat2,how="left",on=["cat2","periodo"])
    df_sellout = df_sellout.merge(df_cat3,how="left",on=["cat3","periodo"])
    print(len(df_sellout))
    
    df_sellout.to_csv(prefijo_arch_salida + str(i) + ".csv", index=False)

Cluster 0
867137
867137
Cluster 1
625413
625413
Cluster 2
229629
229629
Cluster 3
1736970
1736970
Cluster 4
199109
199109
Cluster 5
769474
769474
Cluster 6
246675
246675
Cluster 7
187185
187185
Cluster 8
87708
87708
Cluster 9
142584
142584
Cluster 10
331448
331448
Cluster 11
37950
37950
Cluster 12
418835
418835
Cluster 13


In [ ]:
df_sellout

## Verificaciones

In [ ]:
df_sellout.info()

In [ ]:
df_sellout.isna().sum()

In [ ]:
df_sellout.columns

In [ ]:
len(df_sellout.columns)